# レポート１(提出期限4/29)の解答例

**解答例**

この解答例は、あくまでサンプルです、できるだけ講義の内容と関連付けるようにしました(**第X回講義PYY**)

岩政

## 1.1 線形回帰(10点)

seaborn組み込みのダイアモンドの価格データをつかって重回帰分析を行い、
適切だと思う説明変数の組み合わせをもとめなさい

In [ ]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline

# データの取得・確認
diamond = sns.load_dataset('diamonds')
diamond

In [ ]:
diamond.dtypes

In [ ]:
diamond.describe()

分離してみる

訓練データとテストデータに分割してみます。(**第７回講義P7、ホールドアウトと交差検証**)

In [ ]:
from sklearn.model_selection import train_test_split
x = diamond.drop("price",axis=1)
y = diamond.pop("price")

x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

とりあえず全部のパラメータをいれてみる。

In [ ]:
df_train = pd.concat([x_train, y_train], axis = 1)

In [ ]:
#　回帰の実施

import statsmodels.formula.api as smf
model = smf.glm('price ~ carat + color + clarity +depth+table+cut+x+y+z', df_train) 
fit = model.fit()                                
fit.summary2()


cut,color,clarityはカテゴリ変数として認識してくれている鵜です。

p値から y,zを外します

In [ ]:
model = smf.glm('price ~ carat + color + clarity +depth+table+cut+x', df_train) 
fit = model.fit()                                
fit.summary2()

AICが小さくなるうな説明変数を探します、ステップワイズ法みたいに、要素を１つづつ削ります。(**第６回講義P13、ステップワイズ法**)

In [ ]:
xs = ['carat', 'color', 'clarity','depth','table','cut','x']

便利関数

In [ ]:
def calc_aic(lst):
    for key in lst:
        xs_copy=lst.copy()
        xs_copy.remove(key)
        model = smf.glm('price ~ '+'+'.join(xs_copy), df_train) 
        fit = model.fit()
        print('AIC={},formula={}'.format(fit.aic,'+'.join(xs_copy)))


In [ ]:
calc_aic(xs)

'table'がいらない用です

In [ ]:
calc_aic( ['carat','color', 'clarity','depth','cut','x'])

あまり改善されませんでした

In [ ]:
#carat+color+clarity+depth+cut

model = smf.glm('price ~ carat+color+clarity+depth+cut+x', df_train) 
fit = model.fit()
fit.aic    

次に、テストデータで決定係数R2をつかって評価します（**第５回講義P27**、決定係数)

決定係数$R^2$(coefficient of determination):
$$ R^2 = 1 - \frac{\sum_{i=1}^{n}( y_i - \hat{y_i})^2}{\sum_{i=1}^{n}(y_i - \overline{y})^2} $$

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
# predicting the accuracy score
y_prediction = fit.predict(x_test)
score=r2_score(y_test,y_prediction)
print('r2 socre is ',score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_prediction))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_prediction)))

r2スコアで変数選択してみます。

In [ ]:
def calc_r2score(lst):
    for key in lst:
        xs_copy=lst.copy()
        xs_copy.remove(key)
        model = smf.glm('price ~ '+'+'.join(xs_copy), df_train) 
        fit = model.fit()
        y_prediction = fit.predict(x_test)
        score=r2_score(y_test,y_prediction)
        print('R2Score={},formula={}'.format(score,'+'.join(xs_copy)))

In [ ]:
calc_r2score(xs)

やっぱりtableがいらないようです。

In [ ]:
calc_r2score( ['carat','color', 'clarity','depth','cut','x'])

In [ ]:
model = smf.glm('price ~ carat+color+clarity+depth+cut+x', df_train) 
fit = model.fit()

訓練データをプロット

In [ ]:
y_pred = fit.predict(x_train)
plt.scatter(y_pred,y_train)

テストデータで性能を評価

In [ ]:
y_pred = fit.predict(x_test)
plt.scatter(y_pred,y_test)

In [ ]:
#予測とデータのお比較
#diamond['predict'] = fit.predict(x_test) #予測を行い，結果を'predict'列に追加
#diamond.plot.scatter(x='predict',y='price'); #描画

## 1.2 ロジスティック回帰(10点)

seabornの組み込みデータtitnaicを用いて、生存者をロジスティック回帰を実施しなさい

In [ ]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline

# データの取得・確認
dataset = sns.load_dataset('titanic')
dataset

前処理は以下を用い、データセットはdataset1を利用してもよい。独自に前処理を行ってもよい

In [ ]:
# 重複しているカラムと影響の少ない変数の削除
df = ["sibsp","parch","fare","embarked","class","who","adult_male","deck","embark_town","alive","alone"]
dataset1 = dataset.drop(df,axis = 1)
dataset1.head()

# 欠損値の確認
dataset1.isnull().sum()

# 欠損値に平均を補完し性別をダミー変数に変更（男性:1,女性:0）
dataset1["age"] = dataset1["age"].fillna(dataset1["age"].mean())
dataset1["sex"] = dataset1["sex"].map({"male":1, "female":0})
dataset1.isnull().sum()


In [ ]:
dataset1

In [ ]:
sns.pairplot(dataset1)

トレーニング用とテスト用に分割します

In [ ]:
x = dataset1.drop("survived",axis=1)
y = dataset1.pop("survived")
x_train, x_test , y_train, y_test = train_test_split(x,y,test_size = 0.1, random_state = 1)

In [ ]:
df_train = pd.concat([x_train, y_train], axis = 1)

In [ ]:
#　回帰の実施

import statsmodels.api as sm
model = smf.glm(formula="survived ~ sex + pclass + age", data=df_train, family= sm.families.Binomial() )
res = model.fit() #学習
print(res.summary2())

In [ ]:
#dataset1['predict'] = res.predict() #予測を行い，結果を'predict'列に追加
#dataset1.plot.scatter(x='predict',y='survived'); #描画
y_pred = res.predict(x_test)
plt.scatter(y_pred,y_test)
score=r2_score(y_test,y_pred)
print('AIC={}, r2 socre ={} '.format(res.aic,score))

そうえいば、plcassもsexもカテゴリー変数だったので、カテゴリ変数であることを明示的にformulaに記入。(**第６回講義P14、カテゴリー変数の取り扱い**)

In [ ]:
model = smf.glm(formula="survived ~ C(sex) + C(pclass) + age", data=df_train, family= sm.families.Binomial() )
res = model.fit() #学習
print(res.summary2())

ちょっとAICが上がったかな。

In [ ]:
#dataset1['predict'] = res.predict() #予測を行い，結果を'predict'列に追加
#dataset1.plot.scatter(x='predict',y='survived'); #描画

y_pred = res.predict(x_test)
plt.scatter(y_pred,y_test)
score=r2_score(y_test,y_pred)
print('r2 socre is ',score)
print('AIC={}, r2 socre ={} '.format(res.aic,score))

ageと性別(sex)で傾向をみている

In [ ]:
def get_color(sex):
    if(sex==0):
        return 'orange'
    return 'blue'
colors=[get_color(f) for f in x_test['sex']]
plt.scatter(x_test['age'],y_test,c=colors)#
plt.scatter(x_test['age'],y_pred,c=colors,alpha=0.5)

なんとなく、女性(sex=0)で若いほうが生存率が高いのか、、男女差もありそうだな。

In [ ]:
plt.hist(y_pred)

In [ ]:
plt.hist(y_test)

In [ ]:
res.params

seabornの lmplotを使うと、ロジスティック回帰をした上に、可視化まで一度にやってくれる模様(**第３回講義P25、Implotによる可視化**)

ここでは男女別で、年齢による生存率をロジスティック回帰をやってみた

In [ ]:
sns.lmplot(x="age", y="survived", hue='sex',data=df_train, logistic=True)

男性(sex=1)と女性(sex=0)では傾向が違うことがわかる。。。